## Persistence In LangGraph

In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage, BaseMessage
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv


load_dotenv()

True

In [2]:
hugging_face_llm = HuggingFaceEndpoint(
    # repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    task="conversational", 
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)
llm = ChatHuggingFace(llm=hugging_face_llm)

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.',
 'explanation': 'The joke "Why did the pizza go to therapy? Because it was feeling crusty" is a play on words, relying on the double meaning of the term "crusty."\n\nIn this context, the word "crusty" has a literal meaning, referring to the crunchy outer layer of a pizza, known as the crust. However, the phrase "feeling crusty" also has an idiomatic meaning, equivalent to feeling irritable, grumpy, or having a short temper.\n\nThe punchline of the joke is a clever twist on the expected answer, as the listener is initially led to believe that the pizza is seeking therapy due to being literally made of crust. Instead, the joke subverts this expectation with a more humorous and relatable reason, implying that the pizza is feeling upset or annoyed, much like a person who might visit a therapist for emotional issues.\n\nThe joke relies on the listener making the connection between the literal 

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.', 'explanation': 'The joke "Why did the pizza go to therapy? Because it was feeling crusty" is a play on words, relying on the double meaning of the term "crusty."\n\nIn this context, the word "crusty" has a literal meaning, referring to the crunchy outer layer of a pizza, known as the crust. However, the phrase "feeling crusty" also has an idiomatic meaning, equivalent to feeling irritable, grumpy, or having a short temper.\n\nThe punchline of the joke is a clever twist on the expected answer, as the listener is initially led to believe that the pizza is seeking therapy due to being literally made of crust. Instead, the joke subverts this expectation with a more humorous and relatable reason, implying that the pizza is feeling upset or annoyed, much like a person who might visit a therapist for emotional issues.\n\nThe joke relies on the listener making the connection b

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.', 'explanation': 'The joke "Why did the pizza go to therapy? Because it was feeling crusty" is a play on words, relying on the double meaning of the term "crusty."\n\nIn this context, the word "crusty" has a literal meaning, referring to the crunchy outer layer of a pizza, known as the crust. However, the phrase "feeling crusty" also has an idiomatic meaning, equivalent to feeling irritable, grumpy, or having a short temper.\n\nThe punchline of the joke is a clever twist on the expected answer, as the listener is initially led to believe that the pizza is seeking therapy due to being literally made of crust. Instead, the joke subverts this expectation with a more humorous and relatable reason, implying that the pizza is feeling upset or annoyed, much like a person who might visit a therapist for emotional issues.\n\nThe joke relies on the listener making the connection 

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti go to therapy? \n\nIt was feeling a little "twisted."',
 'explanation': 'This joke is a play on words, using a common idiomatic expression to create a pun. The phrase "feeling a little twisted" has two separate meanings:\n\n1.  In a literal sense, spaghetti can be twisted or turned while it\'s being cooked to create a certain texture or shape.\n2.  In an idiomatic sense, "feeling twisted" is a common phrase used to describe someone who is experiencing mental health issues or emotional distress, often indicated by feeling confused, anxious, or unstable.\n\nThe joke relies on the clever use of a double meaning to create humor. It takes the common phrase "feeling a little twisted" and applies it to a situation where the subject (spaghetti) can literally be twisted, but also implies that it\'s experiencing emotional distress, hence the need for therapy.\n\nThis type of wordplay is a classic example of a pun, which is a form of humor that e

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.', 'explanation': 'The joke "Why did the pizza go to therapy? Because it was feeling crusty" is a play on words, relying on the double meaning of the term "crusty."\n\nIn this context, the word "crusty" has a literal meaning, referring to the crunchy outer layer of a pizza, known as the crust. However, the phrase "feeling crusty" also has an idiomatic meaning, equivalent to feeling irritable, grumpy, or having a short temper.\n\nThe punchline of the joke is a clever twist on the expected answer, as the listener is initially led to believe that the pizza is seeking therapy due to being literally made of crust. Instead, the joke subverts this expectation with a more humorous and relatable reason, implying that the pizza is feeling upset or annoyed, much like a person who might visit a therapist for emotional issues.\n\nThe joke relies on the listener making the connection b

In [ ]:
list(workflow.get_state_history(config1))   

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.', 'explanation': 'The joke "Why did the pizza go to therapy? Because it was feeling crusty" is a play on words, relying on the double meaning of the term "crusty."\n\nIn this context, the word "crusty" has a literal meaning, referring to the crunchy outer layer of a pizza, known as the crust. However, the phrase "feeling crusty" also has an idiomatic meaning, equivalent to feeling irritable, grumpy, or having a short temper.\n\nThe punchline of the joke is a clever twist on the expected answer, as the listener is initially led to believe that the pizza is seeking therapy due to being literally made of crust. Instead, the joke subverts this expectation with a more humorous and relatable reason, implying that the pizza is feeling upset or annoyed, much like a person who might visit a therapist for emotional issues.\n\nThe joke relies on the listener making the connection 

### Time Travel

In [15]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0899ef-681b-6d3c-8001-cf129ed02e55"}})

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0899ef-681b-6d3c-8001-cf129ed02e55'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-09-04T14:53:48.202097+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0899ef-5c27-6e86-8000-b59de6ee2013'}}, tasks=(PregelTask(id='67681e4e-5693-f8b2-89c2-c664c1fd5b4c', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result={'explanation': 'The joke "Why did the pizza go to therapy? Because it was feeling crusty" is a play on words, relying on the double meaning of the term "crusty."\n\nIn this context, the word "crusty" has a literal meaning, referring to the crunchy outer layer of a pizza, known as the crust. However, the phrase "feeling crusty" 

In [16]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0899ef-681b-6d3c-8001-cf129ed02e55"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.',
 'explanation': 'The joke relies on a play on words to create humor. The word "crusty" has a double meaning here:\n\n1.  In a literal sense, pizza dough is crusty, referring to its crispy outer layer.\n2.  In an idiomatic sense, "feeling crusty" is a colloquial expression that means being in a bad mood or irritable, similar to feeling "grumpy" or "testy."\n\nThe joke sets up a scenario where a pizza (an inanimate object) is going to therapy, implying that it has emotional issues. The punchline "because it was feeling crusty" is a clever connection between the pizza\'s physical properties (crusty crust) and the idiomatic expression for being in a bad mood. This unexpected twist creates the humor in the joke, making it a clever play on words.'}

In [17]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.', 'explanation': 'The joke relies on a play on words to create humor. The word "crusty" has a double meaning here:\n\n1.  In a literal sense, pizza dough is crusty, referring to its crispy outer layer.\n2.  In an idiomatic sense, "feeling crusty" is a colloquial expression that means being in a bad mood or irritable, similar to feeling "grumpy" or "testy."\n\nThe joke sets up a scenario where a pizza (an inanimate object) is going to therapy, implying that it has emotional issues. The punchline "because it was feeling crusty" is a clever connection between the pizza\'s physical properties (crusty crust) and the idiomatic expression for being in a bad mood. This unexpected twist creates the humor in the joke, making it a clever play on words.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089a16-87d4-68d2-8002-0b93a292f1d1

### Updating State

In [18]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0899c9-bcaf-63ce-bfff-6da213c76424", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f089a16-d4e6-6a26-8000-d23d98e163f0'}}

In [19]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089a16-d4e6-6a26-8000-d23d98e163f0'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-09-04T15:11:26.508076+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0899c9-bcaf-63ce-bfff-6da213c76424'}}, tasks=(PregelTask(id='230261ad-834a-7678-df79-4dac4ce7e2f6', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy? \n\nBecause it was feeling crusty.', 'explanation': 'The joke relies on a play on words to create humor. The word "crusty" has a double meaning here:\n\n1.  In a literal sense, pizza dough is crusty, referring to its crispy outer layer.\n2.  In an idiomatic sense, "feeling crusty" is a colloquial

In [20]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f089a16-d4e6-6a26-8000-d23d98e163f0"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy? \n\nBecause it was feeling a little "wrapped up" in its emotions.',
 'explanation': 'This joke is a play on words that combines a common idiomatic expression with a literal characteristic of a samosa. \n\nA samosa is a type of Indian or Middle Eastern pastry that consists of a filling wrapped in a thin layer of dough. The phrase "wrapped up" has a broader meaning, implying that someone is overwhelmed or preoccupied with their emotions. \n\nIn this joke, the wordplay lies in the fact that the samosa, being literally wrapped in dough, takes on the idiomatic expression and gives it a new meaning. The punchline is funny because it takes a common expression and applies it to a specific characteristic of the samosa, creating a clever and unexpected connection between the setup and the punchline.'}

In [21]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy? \n\nBecause it was feeling a little "wrapped up" in its emotions.', 'explanation': 'This joke is a play on words that combines a common idiomatic expression with a literal characteristic of a samosa. \n\nA samosa is a type of Indian or Middle Eastern pastry that consists of a filling wrapped in a thin layer of dough. The phrase "wrapped up" has a broader meaning, implying that someone is overwhelmed or preoccupied with their emotions. \n\nIn this joke, the wordplay lies in the fact that the samosa, being literally wrapped in dough, takes on the idiomatic expression and gives it a new meaning. The punchline is funny because it takes a common expression and applies it to a specific characteristic of the samosa, creating a clever and unexpected connection between the setup and the punchline.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089a18-2bc3-653c-80

### Fault Tolerance

In [22]:
import time

# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [27]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(100)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [28]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [29]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [30]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [31]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f089a28-6595-6994-8003-634cafb92ff9'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-04T15:19:18.019506+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f089a28-6593-6996-8002-13337b58c7a1'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f089a28-6593-6996-8002-13337b58c7a1'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-04T15:19:18.018679+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f089a24-542e-6de4-8001-c9066ee20cfa'}}, tasks=(PregelTask(id='ffbf4484-54b4-103b-1477-ed863ef7b52